In [ ]:
!nvcc --version  # To check if CUDA is available

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
print('in progress')

in progress


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-dp041geu
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-dp041geu
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10733 sha256=25d7336482e00933626da974c4579936e1bcd2169b147b54a7d099795d56a4e1
  Stored in directory: /tmp/pip-ephem-wheel-cache-otgtrrjw/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
pip install nvcc4jupyter


In [ ]:
%load_ext nvcc4jupyter
%reload_ext nvcc4jupyter



The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpqqjq5_wy".


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/GPU_prog

#%%writefile file_name.cu

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/GPU_prog


In [ ]:
!nvcc -o compile_file_name HW6_Prob5.cu


/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/11/../../../x86_64-linux-gnu/Scrt1.o: in function `_start':
(.text+0x1b): undefined reference to `main'
collect2: error: ld returned 1 exit status


1. _global_ specifier



*   CUDA C++ allows the programmer to define C++ functions, called kernels, that are executed
𝑁
N times in parallel by
𝑁
N different CUDA threads, as opposed to only once like regular C++ functions.
*   A kernel is defined using the __global__ declaration specifier.


*   The number of CUDA threads that execute the kernel for a given kernel call is specified using a new <<<...>>> execution configuration syntax.

*  Each thread that executes the kernel is given a unique thread ID, which is accessible within the kernel through built-in variables.







In [ ]:
%%writefile HW6_Prob5.cu
#include <stdio.h>
#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void mul(int n, float *x, float *y)
{
    for (int i = 0; i < n; i++) {
        y[i] = x[i] * y[i];
    }
}

int main(void)
{
    int N = 1 << 20; // 1048576 number of elements
    float *x, *y;
    cudaEvent_t start, end;

    // Allocate Unified Memory — accessible from CPU or GPU
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));

    // Initialize x and y arrays on the host
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Run kernel on 1M elements on the GPU
    mul<<<1, 1>>>(N, x, y);

    cudaEventRecord(end);
    // Wait for GPU to finish before accessing on host
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);

    // Free memory
    cudaFree(x);
    cudaFree(y);

    printf("The time required: %f", time);
    return 0;
}

Overwriting HW6_Prob5.cu


In [ ]:
!nvcc -o HW6_Prob5 HW6_Prob5.cu #Establish the code

In [ ]:
!./HW6_Prob5

The time required: 108.705757

In [ ]:
!nvprof ./HW6_Prob5 # Code to compile

==7094== NVPROF is profiling process 7094, command: ./HW6_Prob5
The time required: 108.817123==7094== Profiling application: ./HW6_Prob5
==7094== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  108.62ms         1  108.62ms  108.62ms  108.62ms  mul(int, float*, float*)
      API calls:   64.16%  196.03ms         2  98.013ms  44.540us  195.98ms  cudaMallocManaged
                   35.55%  108.63ms         1  108.63ms  108.63ms  108.63ms  cudaEventSynchronize
                    0.15%  458.22us         2  229.11us  223.41us  234.81us  cudaFree
                    0.06%  192.35us         1  192.35us  192.35us  192.35us  cudaLaunchKernel
                    0.04%  130.63us       114  1.1450us     137ns  51.404us  cuDeviceGetAttribute
                    0.01%  40.508us         2  20.254us  1.4530us  39.055us  cudaEventCreate
                    0.01%  23.658us         2  11.829us  3.4060us  20.252us  cudaEventRe

In [ ]:
%%writefile HW6_Prob5.cu
#include <stdio.h>
#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void mul(int n, float *x, float *y)
{
  int index = threadIdx.x;
  int stride = blockDim.x;

    for (int i = index; i < n; i += stride) {
        y[i] = x[i] * y[i];
    }
}

int main(void)
{
    int N = 1 << 20; // 1048576 number of elements
    float *x, *y;
    cudaEvent_t start, end;

    // Allocate Unified Memory — accessible from CPU or GPU
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));

    // Initialize x and y arrays on the host
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Run kernel on 1M elements on the GPU
    mul<<<1, 256>>>(N, x, y);

    cudaEventRecord(end);
    // Wait for GPU to finish before accessing on host
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);

    // Free memory
    cudaFree(x);
    cudaFree(y);

    printf("The time required: %f", time);
    return 0;
}

Overwriting HW6_Prob5.cu


In [ ]:
!nvcc -o HW6_Prob5 HW6_Prob5.cu #Establish the code

In [ ]:
!./HW6_Prob5

The time required: 3.952736

In [ ]:
!nvprof ./HW6_Prob5 # Code to compile

==7259== NVPROF is profiling process 7259, command: ./HW6_Prob5
==7259== Profiling application: ./HW6_Prob5
The time required: 4.125920==7259== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.9026ms         1  3.9026ms  3.9026ms  3.9026ms  mul(int, float*, float*)
      API calls:   97.73%  205.67ms         2  102.83ms  41.854us  205.62ms  cudaMallocManaged
                    1.86%  3.9105ms         1  3.9105ms  3.9105ms  3.9105ms  cudaEventSynchronize
                    0.20%  424.67us         2  212.33us  204.88us  219.79us  cudaFree
                    0.10%  203.95us         1  203.95us  203.95us  203.95us  cudaLaunchKernel
                    0.08%  164.41us       114  1.4420us     135ns  84.838us  cuDeviceGetAttribute
                    0.01%  21.718us         2  10.859us  3.7600us  17.958us  cudaEventRecord
                    0.01%  18.448us         2  9.2240us     987ns  17.461us  cudaEventCrea

In [ ]:
%%writefile HW6_Prob5.cu
#include <stdio.h>
#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void mul(int n, float *x, float *y)
{
  int index = blockDim.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

    for (int i = index; i < n; i += stride) {
        y[i] = x[i] * y[i];
    }
}

int main(void)
{
    int N = 1 << 20; // 1048576 number of elements
    float *x, *y;
    cudaEvent_t start, end;
    int blockSize = 256;
    int numBlocks = (N + blockSize - 1) / blockSize;

    // Allocate Unified Memory — accessible from CPU or GPU
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));

    // Initialize x and y arrays on the host
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Run kernel on 1M elements on the GPU
    mul<<<numBlocks, blockSize>>>(N, x, y);

    cudaEventRecord(end);
    // Wait for GPU to finish before accessing on host
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);

    // Free memory
    cudaFree(x);
    cudaFree(y);

    printf("The time required: %f", time);
    return 0;
}

Overwriting HW6_Prob5.cu


In [ ]:
!nvcc -o HW6_Prob5 HW6_Prob5.cu #Establish the code

In [ ]:
!./HW6_Prob5

The time required: 0.711520

In [ ]:
!nvprof ./HW6_Prob5 # Code to compile

==7393== NVPROF is profiling process 7393, command: ./HW6_Prob5
The time required: 0.806912==7393== Profiling application: ./HW6_Prob5
==7393== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  572.83us         1  572.83us  572.83us  572.83us  mul(int, float*, float*)
      API calls:   99.22%  218.00ms         2  109.00ms  56.097us  217.94ms  cudaMallocManaged
                    0.28%  625.83us         1  625.83us  625.83us  625.83us  cudaEventSynchronize
                    0.26%  562.34us         2  281.17us  259.08us  303.26us  cudaFree
                    0.11%  231.28us         1  231.28us  231.28us  231.28us  cudaLaunchKernel
                    0.08%  171.60us       114  1.5050us     155ns  64.212us  cuDeviceGetAttribute
                    0.02%  42.522us         2  21.261us  3.7420us  38.780us  cudaEventRecord
                    0.02%  39.180us         2  19.590us  4.7750us  34.405us  cudaEventCrea

In [ ]:
%%writefile HW6_Prob5.cu
#include <stdio.h>
#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void mul(int n, float *x, float *y)
{
  int index = blockDim.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

    for (int i = index; i < n; i += stride) {
        y[i] = x[i] * y[i];
    }
}

int main(void)
{
    int N = 1 << 20; // 1048576 number of elements
    float *x, *y;
    cudaEvent_t start, end;
    int blockSize = 1024;
    int numBlocks = (N + blockSize - 1) / blockSize;

    // Allocate Unified Memory — accessible from CPU or GPU
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));

    // Initialize x and y arrays on the host
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Run kernel on 1M elements on the GPU
    mul<<<numBlocks, blockSize>>>(N, x, y);

    cudaEventRecord(end);
    // Wait for GPU to finish before accessing on host
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);

    // Free memory
    cudaFree(x);
    cudaFree(y);

    printf("The time required: %f", time);
    return 0;
}

Overwriting HW6_Prob5.cu


In [ ]:
!nvcc -o HW6_Prob5 HW6_Prob5.cu #Establish the code

In [ ]:
!./HW6_Prob5

The time required: 0.186400

In [ ]:
!nvprof ./HW6_Prob5 # Code to compile

==7648== NVPROF is profiling process 7648, command: ./HW6_Prob5
The time required: 0.236992==7648== Profiling application: ./HW6_Prob5
==7648== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  25.664us         1  25.664us  25.664us  25.664us  mul(int, float*, float*)
      API calls:   99.63%  202.77ms         2  101.39ms  42.377us  202.73ms  cudaMallocManaged
                    0.15%  303.80us         2  151.90us  133.17us  170.63us  cudaFree
                    0.10%  208.95us         1  208.95us  208.95us  208.95us  cudaLaunchKernel
                    0.06%  130.70us       114  1.1460us     138ns  51.921us  cuDeviceGetAttribute
                    0.02%  37.593us         1  37.593us  37.593us  37.593us  cudaEventSynchronize
                    0.01%  23.591us         2  11.795us  5.1560us  18.435us  cudaEventCreate
                    0.01%  21.263us         2  10.631us  3.6260us  17.637us  cudaEventReco

In [ ]:
%%writefile HW6_Prob5.cu
#include <stdio.h>
#include <iostream>
#include <math.h>

// Kernel function to add the elements of two arrays
__global__
void mul(int n, float *x, float *y)
{
  int index = blockDim.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;

    for (int i = index; i < n; i += stride) {
        y[i] = x[i] * y[i];
    }
}

int main(void)
{
    int N = 1 << 20; // 1048576 number of elements
    float *x, *y;
    cudaEvent_t start, end;
    int blockSize = 2048;
    int numBlocks = (N + blockSize - 1) / blockSize;

    // Allocate Unified Memory — accessible from CPU or GPU
    cudaMallocManaged(&x, N * sizeof(float));
    cudaMallocManaged(&y, N * sizeof(float));

    // Initialize x and y arrays on the host
    for (int i = 0; i < N; i++) {
        x[i] = 1.0f;
        y[i] = 2.0f;
    }

    cudaEventCreate(&start);
    cudaEventCreate(&end);
    cudaEventRecord(start);

    // Run kernel on 1M elements on the GPU
    mul<<<numBlocks, blockSize>>>(N, x, y);

    cudaEventRecord(end);
    // Wait for GPU to finish before accessing on host
    cudaEventSynchronize(end);

    float time = 0;
    cudaEventElapsedTime(&time, start, end);

    // Free memory
    cudaFree(x);
    cudaFree(y);

    printf("The time required: %f", time);
    return 0;
}

Overwriting HW6_Prob5.cu


In [ ]:
!nvcc -o HW6_Prob5 HW6_Prob5.cu #Establish the code

In [ ]:
!./HW6_Prob5

The time required: 0.153728

In [ ]:
!nvprof ./HW6_Prob5 # Code to compile

==8848== NVPROF is profiling process 8848, command: ./HW6_Prob5
==8848== Profiling application: ./HW6_Prob5
==8848== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   99.61%  253.18ms         2  126.59ms  51.968us  253.13ms  cudaMallocManaged
                    0.18%  467.73us         2  233.87us  226.27us  241.47us  cudaFree
                    0.09%  236.52us         1  236.52us  236.52us  236.52us  cudaLaunchKernel
                    0.07%  186.74us       114  1.6380us     198ns  71.955us  cuDeviceGetAttribute
                    0.01%  30.391us         2  15.195us  6.3410us  24.050us  cudaEventCreate
                    0.01%  29.384us         2  14.692us  4.9890us  24.395us  cudaEventRecord
                    0.00%  12.597us         1  12.597us  12.597us  12.597us  cuDeviceGetName
                    0.00%  7.3680us         1  7.3680us  7.3680us  7.3680us  cuDeviceGetPCIBusId
        